## Tic-Tac-Toe Minimax

In [ ]:
import math

# --- Game Board and Utilities ---

def create_board():
    """Creates an empty 3x3 Tic-Tac-Toe board."""
    return [[' ' for _ in range(3)] for _ in range(3)]

def print_board(board):
    """Prints the Tic-Tac-Toe board to the console."""
    print("\nCurrent board:")
    for i, row in enumerate(board):
        print(" " + " | ".join(row))
        if i < 2:
            print("---------")

def get_empty_cells(board):
    """Returns a list of (row, col) tuples for empty cells."""
    cells = []
    for r in range(3):
        for c in range(3):
            if board[r][c] == ' ':
                cells.append((r, c))
    return cells

def check_winner(board, player_x='X', player_o='O'):
    """
    Checks if there is a winner or a draw.
    Returns: 'X' if X wins, 'O' if O wins, 'Draw' if it's a draw, None otherwise.
    """
    # Check rows
    for row in board:
        if row[0] == row[1] == row[2] and row[0] != ' ':
            return row[0]
    # Check columns
    for col in range(3):
        if board[0][col] == board[1][col] == board[2][col] and board[0][col] != ' ':
            return board[0][col]
    # Check diagonals
    if board[0][0] == board[1][1] == board[2][2] and board[0][0] != ' ':
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] and board[0][2] != ' ':
        return board[0][2]
    # Check for draw (no empty cells left)
    if not get_empty_cells(board):
        return 'Draw'
    # Game is not over
    return None

# --- Minimax Algorithm ---

def minimax(board, depth, is_maximizing, player_x='X', player_o='O'):
    """
    Minimax algorithm implementation.
    Returns the score of the board state from the perspective of player_x.
    +1 for X win, -1 for O win, 0 for Draw.
    """
    winner = check_winner(board, player_x, player_o)
    if winner == player_x:
        return 1
    elif winner == player_o:
        return -1
    elif winner == 'Draw':
        return 0
    # else: game is not over, continue recursion

    possible_moves = get_empty_cells(board)

    if is_maximizing: # 'X's turn (wants to maximize score)
        best_score = -math.inf
        for r, c in possible_moves:
            board[r][c] = player_x
            score = minimax(board, depth + 1, False, player_x, player_o)
            board[r][c] = ' ' # Undo move
            best_score = max(score, best_score)
        return best_score
    else: # 'O's turn (wants to minimize score)
        best_score = math.inf
        for r, c in possible_moves:
            board[r][c] = player_o
            score = minimax(board, depth + 1, True, player_x, player_o)
            board[r][c] = ' ' # Undo move
            best_score = min(score, best_score)
        return best_score

def get_best_move(board, player):
    """
    Determines the best move for the given player using the minimax algorithm.
    Args:
        board: The current state of the 3x3 game board.
        player: The player ('X' or 'O') whose turn it is.
    Returns:
        A tuple (row, col) representing the best move (0-based index).
    """
    possible_moves = get_empty_cells(board)
    best_move = (-1, -1) # Placeholder

    if player == 'X': # Maximizing player
        best_score = -math.inf
        for r, c in possible_moves:
            board[r][c] = 'X'
            score = minimax(board, 0, False) # Evaluate for O's turn next
            board[r][c] = ' ' # Undo move
            if score > best_score:
                best_score = score
                best_move = (r, c)
            # Optional: Add randomness for equally good moves
            # elif score == best_score:
            #     if random.choice([True, False]):
            #          best_move = (r,c)

    else: # Minimizing player 'O'
        best_score = math.inf
        for r, c in possible_moves:
            board[r][c] = 'O'
            score = minimax(board, 0, True) # Evaluate for X's turn next
            board[r][c] = ' ' # Undo move
            if score < best_score:
                best_score = score
                best_move = (r, c)
            # Optional: Add randomness for equally good moves
            # elif score == best_score:
            #     if random.choice([True, False]):
            #          best_move = (r,c)

    # If multiple moves have the same best score, minimax might just return the first one found.
    # If no moves are possible (shouldn't happen if called before game end), return placeholder.
    # If all moves lead to the same score (e.g., forcing a draw), the first one checked will be returned.
    if best_move == (-1,-1) and possible_moves: # Ensure a move is returned if possible
         return possible_moves[0] # Default to first available move if scores were somehow equal/problematic
    return best_move


# --- Human Player Input ---

def get_human_move(board, player):
    """Gets and validates human player's move input."""
    while True:
        try:
            move = input(f"Enter your move (row column): ")
            row, col = map(int, move.split())

            if 1 <= row <= 3 and 1 <= col <= 3:
                # Convert 1-based input to 0-based index
                r, c = row - 1, col - 1
                if board[r][c] == ' ':
                    print(f"You played at ({row},{col})")
                    return (r, c)
                else:
                    print("Cell already occupied. Try again.")
            else:
                print("Invalid input. Row and column must be between 1 and 3.")
        except ValueError:
            print("Invalid input. Please enter row and column as two numbers separated by space (e.g., '1 2').")
        except Exception as e:
            print(f"An unexpected error occurred: {e}. Please try again.")

# --- Main Game Loop ---

def play_game():
    """Runs the Tic-Tac-Toe game loop."""
    board = create_board()
    human_player = None
    ai_player = None

    print("Welcome to Tic-Tac-Toe!")

    # Choose player symbol
    while human_player not in ['X', 'O']:
        choice = input("Do you want to play as X or O? ").upper()
        if choice in ['X', 'O']:
            human_player = choice
            ai_player = 'O' if human_player == 'X' else 'X'
        else:
            print("Invalid choice. Please enter 'X' or 'O'.")

    print(f"You are {human_player}, I am {ai_player}.")

    current_player = 'X' # X always starts

    winner = None
    while winner is None:
        print_board(board)

        if current_player == human_player:
            row, col = get_human_move(board, human_player)
            board[row][col] = human_player
        else: # AI's turn
            print(f"AI ({ai_player}) is thinking...")
            row, col = get_best_move(board, ai_player)
            board[row][col] = ai_player
            # Print AI move using 1-based indexing for user
            print(f"I play at ({row + 1},{col + 1})")

        winner = check_winner(board)

        # Switch player only if game is not over
        if winner is None:
            current_player = ai_player if current_player == human_player else human_player

    # Game Over
    print_board(board)
    if winner == 'Draw':
        print("It's a draw!")
    elif winner == human_player:
        print("Congratulations! You win!")
    else:
        print("AI wins! Better luck next time.")

# --- Start the program ---
if __name__ == "__main__":
    play_game()

Welcome to Tic-Tac-Toe!
You are X, I am O.

Current board:
   |   |  
---------
   |   |  
---------
   |   |  
You played at (2,2)

Current board:
   |   |  
---------
   | X |  
---------
   |   |  
AI (O) is thinking...
I play at (1,1)

Current board:
 O |   |  
---------
   | X |  
---------
   |   |  
Invalid input. Please enter row and column as two numbers separated by space (e.g., '1 2').
Invalid input. Please enter row and column as two numbers separated by space (e.g., '1 2').
Invalid input. Please enter row and column as two numbers separated by space (e.g., '1 2').
Invalid input. Please enter row and column as two numbers separated by space (e.g., '1 2').
Invalid input. Please enter row and column as two numbers separated by space (e.g., '1 2').
Cell already occupied. Try again.
Cell already occupied. Try again.
